In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Load Data

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224,224)),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

In [4]:
dataset_path = "./dataset"

In [5]:
dataset = datasets.ImageFolder(root = dataset_path, transform=image_transforms)
len(dataset)

2300

In [6]:
dataset.classes

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [7]:
num_classes = len(dataset.classes)
num_classes

6

In [8]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size
train_size, val_size

(1725, 575)

In [9]:
from torch.utils.data import random_split
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

## Model Training and Hyperparameter Tunning 

In [11]:
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')

        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.layer4.parameters():
            param.requires_grad = True

        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self,x):
        x = self.model(x)
        return x

In [17]:
def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate',0.2,0.7)

    model = CarClassifierResNet(num_classes=num_classes, dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images,labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    end = time.time()
    print(f"Execution time: {end - start} seconds")

    return accuracy
            
            

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-12-10 08:37:53,012] A new study created in memory with name: no-name-c50d495b-43e5-4026-9653-7f9ae28df0c8
[I 2025-12-10 09:22:48,171] Trial 0 finished with value: 67.47826086956522 and parameters: {'lr': 0.006496407493312184, 'dropout_rate': 0.674368720440464}. Best is trial 0 with value: 67.47826086956522.


Execution time: 2693.8497042655945 seconds


[I 2025-12-10 10:30:18,539] Trial 1 finished with value: 53.04347826086956 and parameters: {'lr': 1.9039028551391383e-05, 'dropout_rate': 0.5995254745378842}. Best is trial 0 with value: 67.47826086956522.


Execution time: 4049.0727322101593 seconds


[I 2025-12-10 10:54:54,938] Trial 2 finished with value: 75.47826086956522 and parameters: {'lr': 0.00017842572481665034, 'dropout_rate': 0.5555195414524581}. Best is trial 2 with value: 75.47826086956522.


...

[I 2025-12-10 19:17:43,771] Trial 14 finished with value: 77.3913043478261 and parameters: {'lr': 0.003002862417669321, 'dropout_rate': 0.3828498040840661}. Best is trial 12 with value: 80.17391304347827.


Execution time: 1035.3093659877777 seconds


[I 2025-12-10 20:06:55,554] Trial 15 finished with value: 78.78260869565217 and parameters: {'lr': 0.0003528339653694736, 'dropout_rate': 0.4894758342110775}. Best is trial 12 with value: 80.17391304347827.


Execution time: 2951.024661064148 seconds


[I 2025-12-10 20:12:34,763] Trial 16 pruned. 
[I 2025-12-10 20:31:35,703] Trial 17 pruned. 
[I 2025-12-10 20:37:21,295] Trial 18 pruned. 
[I 2025-12-10 20:43:27,239] Trial 19 pruned. 


In [14]:
study.best_params

{'lr': 0.0009874530852997485, 'dropout_rate': 0.39975557263960615}